In [4]:
import torch
import numpy as np
from network import C3D_model
import cv2
import time
torch.backends.cudnn.benchmark = True
import os
import shutil

In [8]:
def CenterCrop(frame, size):
    h, w = np.shape(frame)[0:2]
    th, tw = size
    x1 = int(round((w - tw) / 2.))
    y1 = int(round((h - th) / 2.))

    frame = frame[y1:y1 + th, x1:x1 + tw, :]
    return np.array(frame).astype(np.uint8)


def center_crop(frame):
    frame = frame[8:120, 30:142, :]
    return np.array(frame).astype(np.uint8)

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device being used:", device)

with open('./dataloaders/2class.txt', 'r') as f:
    class_names = f.readlines()
#    f.close()
# init model
model = C3D_model.C3D(num_classes=2)
checkpoint = torch.load('/home/seino_tuanpn7/ThaoNguyen/video-recognition/C3D-traindata_epoch-19.pth.tar', map_location=lambda storage, loc: storage)
model.load_state_dict(checkpoint['state_dict'])
model.to(device)
model.eval()

Device being used: cuda:0


C3D(
  (conv1): Conv3d(3, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool1): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool2): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3a): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv3b): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool3): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv4a): Conv3d(256, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv4b): Conv3d(512, 512, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv5a): Conv3d(512, 512, kernel_size=(3, 3, 3), stride=

In [17]:
pathvideos = '/home/seino_tuanpn7/violence/data/violence-data'
pathtest = '/home/seino_tuanpn7/ThaoNguyen/video-recognition/traindata/train'
pathval = '/home/seino_tuanpn7/ThaoNguyen/video-recognition/traindata/val'
video_extensions = ['.mp4', '.avi', '.mpg', '.mov']
y_true = []
y_pred = []
foldername = os.listdir(pathtest)
for n in foldername:
    filename = os.listdir(pathtest + '/' + n)
    print(n)
    for name in filename:
        for ext in video_extensions:
            video = pathvideos + '/' + n + '/' + name + ext
            cap = cv2.VideoCapture(video)
            if cap.isOpened():
                video_paths = video
                y_true.append(n)
#                 print(video)
        cap = cv2.VideoCapture(video_paths)
        retaining = True

        clip = []

        while retaining:
            start = time.time()
            retaining, frame = cap.read()
            if not retaining and frame is None:
                continue
            tmp_ = center_crop(cv2.resize(frame, (171, 128)))
            tmp = tmp_ - np.array([[[90.0, 98.0, 102.0]]])
            clip.append(tmp)
            if len(clip) == 16:
                inputs = np.array(clip).astype(np.float32)
                inputs = np.expand_dims(inputs, axis=0)
                inputs = np.transpose(inputs, (0, 4, 1, 2, 3))
                inputs = torch.from_numpy(inputs)
                inputs = torch.autograd.Variable(inputs, requires_grad=False).to(device)
                with torch.no_grad():
                    outputs = model.forward(inputs)

                probs = torch.nn.Softmax(dim=1)(outputs)
                label = torch.max(probs, 1)[1].detach().cpu().numpy()[0]

                cv2.putText(frame, class_names[label].split(' ')[-1].strip(), (20, 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                            (0, 0, 255), 1)
                cv2.putText(frame, "prob: %.4f" % probs[0][label], (20, 40),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                            (0, 0, 255), 1)
                clip.pop(0)

            cv2.imshow('result', frame)
            cv2.waitKey(5)
        cap.release()
        cv2.destroyAllWindows()




if __name__ == '__main__':
    main()

violence


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [1]:
from sklearn.metrics import f1_score, precision_score, recall_score
y_true= [1,1,0,0,1]
y_pred= [1,1,1,1,1]

In [4]:
print(len(y_true)-len(y_pred))
print(f1_score(y_true,y_pred))
print(recall_score(y_true,y_pred))
print(precision_score(y_true,y_pred))

0
0.7499999999999999
1.0
0.6


In [ ]:
sklearn.metrics.f1_score(y_true, y_pred)